In [2]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

# Beep(0.1)
# time.sleep(0.25)
# Beep(0.1)

In [3]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

In [6]:
import time
detections = model(camera.value)

while str(detections) == '[[]]':
    print('no detection')
    time.sleep(1)
    detections = model(camera.value)
    
print(detections)

no detection
no detection
no detection
no detection
no detection
no detection
no detection
no detection
no detection
no detection


KeyboardInterrupt: 

In [17]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

Textarea(value='[[]]')

In [8]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

IndexError: list index out of range

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    #Image zoomed to 224,224 versus 224,244 obstacle avoidance model
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
import PID

global follow_speed
follow_speed = Speed # 기본속력 0.7

global turn_gain
turn_gain = 1.5

global follow_speed_pid, follow_speed_pid_model
follow_speed_pid_model = 1

# follow_speed_pid = PID.PositionalPID(3, 0, 0)
follow_speed_pid = PID.PositionalPID(1.5, 0, 0.05)

global turn_gain_pid
turn_gain_pid = PID.PositionalPID(0.15, 0, 0.05)

global object_yservo_pid
object_yservo_pid = PID.PositionalPID(3, 0.5, 0)

In [ ]:
detectedPerson = 0 # index 1

detectedDog = 0 # 18
detectStop = 0 # 13

detectedCup = 0 # 47
detectedFruit = 0 # 52, 53, 55
detectedBook = 0 # 84

MODE_NONE = 0
MODE_PERSON = 1
MODE_FOLLOW = 2
MODE_CUP = 3
MODE_FRUIT = 4
MODE_BOOK = 5

MIN_KEEP_MODE_TIME = 2.0
actionMode = MODE_NONE
modeTime = time.time()   # 현재 모드로 전환된 시간

# 첫 실행될 때 바로 첫번째 검출 객체에 대한 감성동작이 실행되는 것을 방지하기 위해
# 객체들의 마지막 검출시각을 모두 현재 시각으로 설정
LastTimeDetectedPerson = LastTimeDetectedFruit = LastTimeDetectedCup = modeTime
LastTimeDetectedBook = StartTimeDetectedNone = modeTime

# 동일한 감성동작이 연속해서 반복되지 않도록 마지막 동작 기억
LastPerformedAct = MODE_NONE

In [ ]:
from jetbot import bgr8_to_jpeg

if runMode < 3:
    image_widget = widgets.Image(format='jpeg', width=300, height=300)
    label_widget = widgets.IntText(value=1, description='target#') # person : 1
    detected_widget = widgets.Text(value='', description='detected#')

    display(widgets.VBox([
        widgets.HBox([image_widget]),
        label_widget, detected_widget
    ]))

In [ ]:
def detection_center(detection):
    """Calculate the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Calculate the length of a two-dimensional vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Find the detection closest to the center of the image"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

In [ ]:
def execute(change):
    
    global follow_speed
    global turn_gain
    global follow_speed_pid
    
    target_value_speed = 0
    
    #Update image value
    image = change['new']
    
    global ActTimePeriodPerson
    global LastTimeDetectedPerson
    global ActTimePeriodFruit
    global LastTimeDetectedFruit
    global ActTimePeriodCup
    global LastTimeDetectedCup
    global ActTimePeriodBook
    global LastTimeDetectedBook
    global ActTimePeriodFruit

    global ActTimePeriodNone
    global StartTimeDetectedNone
    global LastPerformedAct
    
    global runMode
    global actionMode
    global modeTime
    
    global detectedPerson
    global detectedDog
    global detectStop
    global detectedCup
    global detectedFruit
    global detectedBook
    
    # 검출된 내용물 받음
    detections = model(image)
    
    # 물체 확인
    for det in detections[0]:
        bbox = det['bbox']
        
        bDrawBox = 1 # 일단 검출 박스를 그림
        
        if det['label'] == 1:
            detectedPerson += 1
        elif det['label'] == 18:
            detectedDog += 1
        elif det['label'] == 13:
            detectedStop += 1
        elif det['label'] == 47:
            detectedCup += 1
        elif (det['label'] == 52) or (det['label'] == 53) or (det['label'] == 55):
            detectedFruit += 1
        elif det['label'] == 84:
            detectedBook += 1
        else:
            bDrawBox = 0 # 원하는 객체 아니면 box 안그림
            
        if runMode < 2:
            if bDrawBox :
                cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), 
                              (int(300 * bbox[2]), int(300 * bbox[3])), (255, 0, 0), 2)
                
        if runMode < 2:
            detects = ''
            if detectedPerson:
                detects += 'Person '
            if detectedDog:
                detects += 'Dog '
            if detectStop:
                detects += 'Stop '
            if detectedCup:
                detects += 'Cup '
            if detectedBook:
                detects += 'Book'
            detected_widget.value = detects
            
            # 어떤 물체가 검출되었을 때, 이전의 물체 검출시각으로 부터 Min 시간이 지나야 전환 가능
            # 모드가 바뀌면, 마지막모드 전환 시각으로부터 특정 시간이 경과하면 해당 동작 1회 실행
            
            newTime = time.time() # 시간 비교 위해
            
        if detectedCup:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_CUP) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedCup > ActTimePeriodCup:
                    if LastPerformedAct != MODE_CUP:
                        ActModeCup()
                        LastPerformedAct = MODE_CUP
                modeTime = LastTimeDetectedCup = newTime
                actionMode = MODE_CUP
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("CUP : " + str(detectedCup))
                        
        elif detectedFruit:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_FRUIT) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedFruit > ActTimePeriodFruit:
                    if LastPerformedAct != MODE_FRUIT:
                        ActModeCup()
                        LastPerformedAct = MODE_FRUIT
                modeTime = LastTimeDetectedFruit = newTime
                actionMode = MODE_FRUIT
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("FRUIT : " + str(detectedFruit))

        elif detectedBook:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_BOOK) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedBook > ActTimePeriodBook:
                    if LastPerformedAct != MODE_BOOK:
                        ActModeCup()
                        LastPerformedAct = MODE_BOOK
                modeTime = LastTimeDetectedBook = newTime
                actionMode = MODE_BOOK
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("BOOK : " + str(detectedBook))
        
        elif detectedDog:
            if (actionMode != MODE_FOLLOW) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                modeTime = newTime
                actionMode = MODE_FOLLOW
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("FOLLOW" + str(detectedDog))
                    
        elif detectedStop:
            if (actionMode != MODE_PERSON) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                modeTime = LastTimeDetectedStop = newTime
                actionMode = MODE_PERSON
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("PERSON!" + str(detectedPerson))
                    
        elif detectedPerson:
            if (actionMode != MODE_FOLLOW) and (actionMode != MODE_PERSON) and (newTime - modeTime > MIN_KEEP_MODE_TIME):
                if newTime - LastTimeDetectedPerson > ActTimePeriodPerson:
                    if LastPerformedAct != MODE_PERSON:
                        ActModeCup()
                        LastPerformedAct = MODE_PERSON
                modeTime = LastTimeDetectedPerson = newTime
                actionMode = MODE_PERSON
                StartTimeDetectedNone = 0
                if runMode < 2:
                    ShowTopInfo("PERSON : " + str(detectedPerson))
                    
        else:
            if actionMode != MODE_FOLLOW:
                
                if actionMode == MODE_NONE:
                    if newTime - LastTimeDetectedPerson > ActTimePeriodNone:
                        if LastPerformedAct != MODE_NONE:
                            ActModeCup()
                            LastPerformedAct = MODE_NONE
                        StartTimeDetectedNone = newTime
                
                else:
                    if newTime - modeTime > MIN_KEEP_MODE_TIME:
                        modeTime = StartTimeDetectedNone = newTime
                        actionMode = MODE_NONE
                        if runMode < 2:
                            ShowTopInfo("(none)")
        
        matching_detections = [d for d in detections[0] if d['label'] == 1]
        
        det = closest_detection(matching_detections)
        
        if det is None:
            Stop() # 검출 안되면 정지
            
        else:
            if runMode < 2:
                bbox = det['bbox']
                cv2.rectangle(image, (int(300 * bbox[0]), int(300 * bbox[1])), 
                              (int(300 * bbox[2]), int(300 * bbox[3])), (0, 255, 0), 4)
            
            if actionMode == MODE_FOLLOW:
                
                # 중심에서 대산 물체의 x축 오프셋 만큼 이동
                center = detection_center(det)
                
                # 따라가는 속도 제어
                follow_speed_pid.SystemOutput = 76000 * (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
                
                follow_speed_pid.SetStepSignal(30000)
                follow_speed_pid.SetInertiaTime(0.2, 0.1)
                
                #Steering gain PID adjustment
                turn_gain_pid.SystemOutput = center[0]
                turn_gain_pid.SetStepSignal(0)
                turn_gain_pid.SetInertiaTime(0.2, 0.1)
                
                target_value_turn_gain = 0.5 + abs(turn_gain_pid.SystemOutput)
                
                if target_value_turn_gain < 0:
                    target_value_turn_gain = 0
                elif target_value_turn_gain > 2:
                    target_value_turn_gain = 2
                    
                # 바퀴 모터 속력
                target_value_speed = 0.5 + follow_speed_pid.SystemOutput / 76000
                
                target_value_speedl = target_value_speed + target_value_turn_gain * center[0]
                target_value_speedr = target_value_speed - target_value_turn_gain * center[0]
                
                if target_value_speedl < 0.4:
                    target_value_speedl = 0
                    
                elif target_value_speedl > 1:
                    target_value_speedl = 1
                    
                if target_value_speedr < 0.4:
                    target_value_speedr = 0
                    
                elif target_value_speedr > 1:
                    target_value_speedr = 1
                
                Move2(target_value_speedl, target_value_speedr)
            
        if runMode < 3:
            # 이미지 디스플레이 업데이트
            image_widget.value = bgr8_to_jpeg(image)
            
### execute 실험!!

modeTime = time.time()
actionMode = MODE_NONE

execute({'new': camera.value})